# Scraping of http://www.parlament.ch

First, we need to scrap some information from the website http://parlament.ch. In this notebook, we will scrap different information. These information will be stored in the folder *data*. If you just cloned the repo and you need some data, please run this python notebook to scrap all the data. 

For the scraping, we are using the library `requests`. The metadata of the website are provided and working with XOData. So, we get the urls using XOData, then we get the XML using `requests` and we transform the XML into JSON using the library `xmltodict`.

URL of the metadata: https://ws.parlament.ch/odata.svc/$metadata

In [20]:
# Import some useful libraries
%matplotlib inline
import pandas as pd
import urllib
import xml.etree.ElementTree as ET
import loader
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Scrap


Tables: Party, Person, Council

In [22]:
df_party = loader.party()
df_person = loader.person()
df_member_council = loader.member_council()



https://ws.parlament.ch/odata.svc/Party?$filter=Language%20eq%20'FR'
https://ws.parlament.ch/odata.svc/Person?$filter=Language%20eq%20'FR'%20and%20ID%20ge%200%20and%20ID%20lt%201000
https://ws.parlament.ch/odata.svc/Person?$filter=Language%20eq%20'FR'%20and%20ID%20ge%201000%20and%20ID%20lt%202000
https://ws.parlament.ch/odata.svc/Person?$filter=Language%20eq%20'FR'%20and%20ID%20ge%202000%20and%20ID%20lt%203000
https://ws.parlament.ch/odata.svc/Person?$filter=Language%20eq%20'FR'%20and%20ID%20ge%203000%20and%20ID%20lt%204000
https://ws.parlament.ch/odata.svc/Person?$filter=Language%20eq%20'FR'%20and%20ID%20ge%204000%20and%20ID%20lt%205000
https://ws.parlament.ch/odata.svc/Person?$filter=Language%20eq%20'FR'%20and%20ID%20ge%205000%20and%20ID%20lt%206000
https://ws.parlament.ch/odata.svc/MemberCouncil?$filter=Language%20eq%20'FR'%20and%20ID%20ge%200%20and%20ID%20lt%201000
https://ws.parlament.ch/odata.svc/MemberCouncil?$filter=Language%20eq%20'FR'%20and%20ID%20ge%201000%20and%20ID%20lt%20

We know that their is 4211 persons registered in database, but API only allows 1000 load at a time.
Let's scrap!

In [49]:
import urllib
def count(table_name):
    url = "https://ws.parlament.ch/odata.svc/" + table_name + "/$count?$filter=Language%20eq%20'FR'"
    with urllib.request.urlopen(url) as response:
        n = response.read()
    # get the number from the bytes
    n = int(str(n).split("'")[1])
    return n
    
    
print(count("Council"))
print(count("Person"))
print(count("MemberCouncil"))
a = count("MemberCouncil")

3
3525
3514


In [19]:
person.head(10)

,DateOfBirth,DateOfDeath,FirstName,GenderAsString,ID,Language,LastName,MaritalStatus,MaritalStatusText,MilitaryRank,...,Modified,NativeLanguage,NumberOfChildren,OfficialName,PersonIdCode,PersonNumber,PlaceOfBirthCanton,PlaceOfBirthCity,Title,TitleText
0,1938-03-02T00:00:00,None,Pierre,m,1,FR,Aguet,2,marié(e),5,...,2015-05-17T21:18:19.387,F,None,Aguet,2200,1,Vaud,Pompaples,None,None
1,1928-02-22T00:00:00,None,Heinz,m,2,FR,Allenspach,None,None,None,...,2015-05-17T21:18:19.387,D,None,Allenspach,2002,2,None,None,None,None
2,1931-01-27T00:00:00,None,Manfred,m,6,FR,Aregger,2,marié(e),7,...,2015-05-17T21:18:19.387,D,5,Aregger,2004,6,Lucerne,Hasle,9,dipl. Bauing. HTL
3,1928-03-04T00:00:00,None,Geneviève,f,7,FR,Aubry,None,None,None,...,2015-05-17T21:18:19.387,F,None,Aubry Geneviève,2005,7,None,None,None,None
4,1947-12-01T00:00:00,None,Rosmarie,f,8,FR,Bär,None,None,None,...,2015-05-17T21:18:19.387,D,None,Bär,2008,8,None,None,None,None
5,1947-11-11T00:00:00,None,Ruedi,m,9,FR,Baumann,2,marié(e),None,...,2015-05-17T21:18:19.387,D,2,Baumann Ruedi,2268,9,Berne,Suberg,10,dipl. Ing. Agr. ETH
6,1942-09-03T00:00:00,None,Peter,m,10,FR,Baumberger,2,marié(e),11,...,2015-05-17T21:18:19.387,D,2,Baumberger Peter,2269,10,Zurich,Winterthur,6,Dr. iur.
7,1938-05-12T00:00:00,None,Ursula,f,11,FR,Bäumlin,2,marié(e),None,...,2015-05-17T21:18:19.387,D,2,Bäumlin Ursula,2011,11,Berne,Berne,12,lic. phil. I
8,1953-03-26T00:00:00,None,Christine,f,12,FR,Beerli,2,marié(e),None,...,2015-05-17T21:18:19.387,D,None,Beerli,2335,12,Berne,Bienne,115,lic. iur.
9,1947-12-02T00:00:00,None,Thierry,m,13,FR,Béguin,2,marié(e),2,...,2015-05-17T21:18:19.387,F,4,Béguin Thierry,2202,13,Neuchâtel,La Chaux-de-Fonds,3,lic. en droit


In [13]:
member_council = loader.member_council()
member_council.head()

https://ws.parlament.ch/odata.svc/MemberCouncil?$filter=Language%20eq%20'FR'%20and%20ID%20ge%200%20and%20ID%20lt%201000
https://ws.parlament.ch/odata.svc/MemberCouncil?$filter=Language%20eq%20'FR'%20and%20ID%20ge%201000%20and%20ID%20lt%202000
https://ws.parlament.ch/odata.svc/MemberCouncil?$filter=Language%20eq%20'FR'%20and%20ID%20ge%202000%20and%20ID%20lt%203000
https://ws.parlament.ch/odata.svc/MemberCouncil?$filter=Language%20eq%20'FR'%20and%20ID%20ge%203000%20and%20ID%20lt%204000
https://ws.parlament.ch/odata.svc/MemberCouncil?$filter=Language%20eq%20'FR'%20and%20ID%20ge%204000%20and%20ID%20lt%205000
https://ws.parlament.ch/odata.svc/MemberCouncil?$filter=Language%20eq%20'FR'%20and%20ID%20ge%205000%20and%20ID%20lt%206000


,Active,AdditionalActivity,AdditionalMandate,BirthPlace_Canton,BirthPlace_City,Canton,CantonAbbreviation,CantonName,Citizenship,Council,...,ParlGroupAbbreviation,ParlGroupFunction,ParlGroupFunctionText,ParlGroupName,ParlGroupNumber,Party,PartyAbbreviation,PartyName,PersonIdCode,PersonNumber
0,false,Réducteur TSV de 1971 à 1983,"Prés. Féd. Romande des Sociolistes ch., Prés. ...",None,Pompaples,22,VD,Vaud,"Sullens (VD),Lutry (VD)",1,...,None,None,None,None,None,12,PSS,Parti socialiste suisse,2200,1
1,false,None,None,None,None,1,ZH,Zurich,"Kreuzlingen (TG),Fällanden (ZH)",1,...,None,None,None,None,None,15,PLR,PLR.Les Libéraux-Radicaux,2002,2
2,false,Zentralpräsident Schweiz. Skliverband 1985 bis...,None,None,Hasle,3,LU,Lucerne,Hasle (LU),1,...,None,None,None,None,None,15,PLR,PLR.Les Libéraux-Radicaux,2004,6
3,false,None,None,None,None,2,BE,Berne,Tavannes (BE),1,...,None,None,None,None,None,None,None,None,2005,7
4,false,None,None,None,None,2,BE,Berne,"Siselen (BE),Richterswil (ZH)",1,...,None,None,None,None,None,None,None,None,2008,8
